In [9]:
import plotly.express as px
import pandas as pd
import os
from datetime import datetime, timedelta
import requests
import json

In [2]:
px.set_mapbox_access_token(os.environ['MAPBOX_TOKEN'])

In [3]:
df = pd.read_csv('data/worldcities.csv').sort_values('population', ascending=False)

In [4]:
lat = '41.316440'
lon = '69.294860'

In [30]:
def make_req(lat, lon, unit_system):
    """
    A vanilla function to make
    API call based on lat, lon
    """
    endpoint = "https://api.climacell.co/v3/weather/realtime"
    params = {
        'lat': lat, 'lon': lon,
        'fields': 'temp', 'unit_system': unit_system,
        'apikey': os.environ['CLIMACELL_API']
    }
    res = requests.request("GET", endpoint, params=params)
    response = json.loads(res.content)
    # Build df
    df_dict = {
        'lat': [lat],
        'lon': [lon],
        'Temperature': [round(response['temp']['value'], 1)],
        'size': [15]
    }
    df = pd.DataFrame(df_dict, index=[0])
    return df

In [39]:
def plot_single(df):
    """
    Vanilla function to 
    plot scatter_mapbox based on single
    location
    """
    # Get time for the moment
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    # Construct the figure
    fig = px.scatter_mapbox(df, hover_data=['Temperature'],
                            lat='lat', lon='lon',
                            size='size',
                            color_continuous_scale=px.colors.cyclical.IceFire,
                            zoom=14)
    fig.update_traces(textposition='top center')
    fig.update_layout(title_text=f'Temperatures for {now}, at ({df["lat"][0]}, {df["lon"][0]})', title_x=0.5)

    return fig

In [40]:
df = make_req(lat, lon, 'si')
df

,lat,lon,Temperature,size
0,41.316440,69.294860,-2.4,15


In [41]:
fig = plot_single(df)
fig.show()